In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks import *
from fastai.script import *

from data_utils import *
from models import *
from learn_utils import *

Available data in test folder: 

- `raw_images`: Raw images before normalization
- `brain_preds`: Brain segmentation predictions from best model
- `skull_stripped_v2`: Skull stripped raw images
- `ventricle_preds`: Ventricle segmentation predictions from best model

### Model and Atlas Sample by Sample Comparison

In [104]:
def dice_score(probs, targs, thresh=0.5):
    hard_preds = probs > thresh
    m1 = hard_preds.flatten()
    m2 = targs.flatten()
    intersection = (m1 * m2).sum()
    return np.round((2. * intersection) / (m1.sum() + m2.sum() + 1e-8),4)

In [125]:
test_path = Path("/home/turgutluk/data/Segmentation_Dataset/CT_Dataset/test2/")

In [126]:
data_name = 'notl_ventricle_ct'
f = data_dict[data_name]
train_paths, valid_paths, test1_paths, test2_paths = f()

In [127]:
brain_preds, brain_targs = pd.read_pickle(test_path/'brain_preds/baseline11_preds.pkl')
ventricle_preds, ventricle_targs = pd.read_pickle(test_path/'ventricle_preds/baseline11_preds.pkl')

In [136]:
model_brain_dice_list = []
atlas_brain_dice_list = []
model_ventricle_dice_list = []
atlas_ventricle_dice_list = []
study_name_list = []

In [137]:
len(test2_paths[0])

18

In [138]:
brain_preds.shape, brain_targs.shape, ventricle_preds.shape, ventricle_targs.shape

(torch.Size([18, 128, 256, 256]),
 torch.Size([18, 128, 256, 256]),
 torch.Size([18, 128, 256, 256]),
 torch.Size([18, 128, 256, 256]))

In [139]:
# get fname
for i in range(len(test2_paths[0])):
    fname = test2_paths[0][i].name
    
    study_name = fname.split("_")[0]
    study_name_list.append(study_name)
    fname, study_name

    brain_pred, brain_targ = to_np(brain_preds[i]), to_np(brain_targs[i])
    atlas_brain_pred = np.load(test_path/f'brain_atlas/{fname}')
    ventricle_pred, ventricle_targ = to_np(ventricle_preds[i]), to_np(ventricle_targs[i])
    atlas_ventricle_pred = np.load(test_path/f'ventricle_atlas/{fname}')

    model_brain_dice = dice_score(brain_pred, brain_targ)
    atlas_brain_dice = dice_score(atlas_brain_pred, brain_targ)
    model_ventricle_dice = dice_score(ventricle_pred, ventricle_targ)
    atlas_ventricle_dice = dice_score(atlas_ventricle_pred, ventricle_targ)

    model_brain_dice_list.append(model_brain_dice)
    atlas_brain_dice_list.append(atlas_brain_dice)
    model_ventricle_dice_list.append(model_ventricle_dice)
    atlas_ventricle_dice_list.append(atlas_ventricle_dice)

In [140]:
df = pd.DataFrame({"study":study_name_list,
                   "model_brain_dice":model_brain_dice_list,
                   "atlas_brain_dice":atlas_brain_dice_list,
                   "model_ventricle_dice":model_ventricle_dice_list,
                   "atlas_ventricle_dice":atlas_ventricle_dice_list})

In [141]:
df

,study,model_brain_dice,atlas_brain_dice,model_ventricle_dice,atlas_ventricle_dice
0,CT-Test-001,0.9799,0.9858,0.7916,0.7710
1,CT-Test-002,0.9793,0.9879,0.7767,0.5548
2,CT-Test-002,0.9782,0.9912,0.8497,0.7916
3,CT-Test-003,0.9828,0.9907,0.7596,0.7670
4,CT-Test-004,0.9749,1.0000,0.4397,1.0000
5,CT-Test-004,0.9812,0.9886,0.6989,0.5216
6,CT-Test-005,0.9796,0.9923,0.8903,0.8541
7,CT-Test-005,0.9826,0.9900,0.7522,0.7396
8,CT-Test-006,0.9780,0.9738,0.8084,0.7808
9,CT-Test-006,0.9820,0.9835,0.8671,0.8630


In [142]:
df.to_csv(test_path/"model_atlas_comparison_CT2.csv", index=False)

### Show volumetric difference

In [170]:
i = 1

In [171]:
fname = test2_paths[0][i].name

study_name = fname.split("_")[0]
study_name_list.append(study_name)
fname, study_name

ventricle_pred, ventricle_targ = to_np(ventricle_preds[i]), to_np(ventricle_targs[i])
ventricle_pred = (ventricle_pred > 0.5).astype(float)
atlas_ventricle_pred = np.load(test_path/f'ventricle_atlas/{fname}')

In [172]:
targ_sum1 = ventricle_targ.sum()
pred_sum1 = ventricle_pred.sum()
atlas_sum1 = atlas_ventricle_pred.sum()

In [173]:
targ_sum1, pred_sum1, atlas_sum1

(39538.0, 27164.0, 15817.0)

In [179]:
i = 2

In [180]:
fname = test2_paths[0][i].name

study_name = fname.split("_")[0]
study_name_list.append(study_name)
fname, study_name

ventricle_pred, ventricle_targ = to_np(ventricle_preds[i]), to_np(ventricle_targs[i])
ventricle_pred = (ventricle_pred > 0.5).astype(float)
atlas_ventricle_pred = np.load(test_path/f'ventricle_atlas/{fname}')

In [181]:
targ_sum2 = ventricle_targ.sum()
pred_sum2 = ventricle_pred.sum()
atlas_sum2 = atlas_ventricle_pred.sum()

In [182]:
targ_sum2, pred_sum2, atlas_sum2

(16856.0, 16074.0, 12670.0)

In [183]:
(targ_sum1 - targ_sum2) / targ_sum1

0.57367593

In [184]:
(pred_sum1 - pred_sum2) / pred_sum1

0.4082609335885731

In [185]:
(atlas_sum1 - atlas_sum2) / pred_sum1

0.11585186275953469